In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
!pip install -U seaborn
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "data/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploring dataset

In [2]:
df=pd.read_csv('data/health-insurance-cross-sell-prediction/train.csv')

In [3]:
df_test=pd.read_csv('data/health-insurance-cross-sell-prediction/test.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [5]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [6]:
df.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000
mean,190555.000000,38.822584,0.997869,26.388807,0.458210,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,15.511611,0.046110,13.229888,0.498251,17213.155057,54.203995,83.671304,0.327936
min,1.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,25.000000,1.000000,15.000000,0.000000,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,36.000000,1.000000,28.000000,0.000000,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,49.000000,1.000000,35.000000,1.000000,39400.000000,152.000000,227.000000,0.000000
max,381109.000000,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


# Preparing data for fitting the model

### Creating Bins

In [7]:
age=pd.cut(df['Age'],bins=5,labels=['A1','A2','A3','A4','A5'])
rc=pd.cut(df['Region_Code'],bins=5,labels=['A1','A2','A3','A4','A5'])
ap=pd.cut(df['Annual_Premium'],bins=5,labels=['A1','A2','A3','A4','A5'])
vin=pd.cut(df['Vintage'],bins=5,labels=['A1','A2','A3','A4','A5'])

In [8]:
age2=pd.cut(df_test['Age'],bins=5,labels=['A1','A2','A3','A4','A5'])
rc2=pd.cut(df_test['Region_Code'],bins=5,labels=['A1','A2','A3','A4','A5'])
ap2=pd.cut(df_test['Annual_Premium'],bins=5,labels=['A1','A2','A3','A4','A5'])
vin2=pd.cut(df_test['Vintage'],bins=5,labels=['A1','A2','A3','A4','A5'])

## Adding columns to our datasets

In [9]:
df['age']=age             
df['rc']=rc 
df['ap']=ap
df['vin']=vin

In [10]:
df_test['age2']=age2       
df_test['rc2']=rc2
df_test['ap2']=ap2
df_test['vin2']=vin2

### Creating dummy columns

In [11]:
df_dum=pd.get_dummies(df[['age','rc','ap','vin','Gender','Vehicle_Age','Vehicle_Damage','Vehicle_Damage']])
df_dum2=pd.get_dummies(df_test[['age2','rc2','ap2','vin2','Gender','Vehicle_Age','Vehicle_Damage','Vehicle_Damage']])

In [12]:
df=pd.concat([df,df_dum],axis=1)
df_test=pd.concat([df_test,df_dum2],axis=1)

In [13]:
df.drop(['Gender','Age','age','Region_Code','rc','Vehicle_Age','Vehicle_Damage','Annual_Premium','ap','Vintage','vin'],axis=1,inplace=True)
df_test.drop(['Gender','Age','age2','Region_Code','rc2','Vehicle_Age','Vehicle_Damage','Annual_Premium','ap2','Vintage','vin2'],axis=1,inplace=True)

# Scaling the data

In [14]:
from sklearn.preprocessing import StandardScaler as sc

In [15]:
scaler=sc()
scaled_df=scaler.fit_transform(df.drop(['id','Response'],axis=1))
scaled_dft=scaler.fit_transform(df_test.drop('id',axis=1))

### Dividing the data

In [16]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte=train_test_split(scaled_df,df['Response'],random_state=42,test_size=0.27)

# Selecting the right model with tuned Hyper-Parameters

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV

In [18]:
sgc=SGDClassifier(penalty='l1',loss='modified_huber',early_stopping=True)
model_params={'alpha':[0.0012,0.0011,0.0013],
              'learning_rate':['invscaling'],
              'max_iter':[690,700,710],
              'validation_fraction':[0.46,0.47,0.48],
              'eta0':[0.65,0.66,0.64]}
ran=RandomizedSearchCV(sgc,param_distributions=model_params,cv=5,n_jobs=-1,verbose=2,n_iter=100)

In [ ]:
ran.fit(scaled_df,df['Response'])

C:\Python310\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [ ]:
ran.best_params_

In [ ]:
ran.best_score_

In [ ]:
y_pred=ran.predict(scaled_dft)

In [ ]:
submissions=pd.DataFrame({'id':df_test['id'],'Response':y_pred})

# Building our Neural-Net model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [ ]:
model=keras.Sequential([layers.Dense(units=1024,activation='relu',input_shape=[32]),
                        layers.BatchNormalization(),
                        layers.Dense(units=1024,activation='relu'),
                        layers.Dropout(0.27),
                        layers.BatchNormalization(),
                        layers.Dense(units=1024,activation='relu'),
                        layers.Dropout(0.27),
                        layers.BatchNormalization(),
                        layers.Dense(units=1024,activation='relu'),
                        layers.Dropout(0.27),
                        layers.BatchNormalization(),
                        layers.Dense(units=1024,activation='relu'),
                        layers.Dropout(0.27),
                        layers.BatchNormalization(),
                        layers.Dense(units=1024,activation='relu'),
                        layers.Dropout(0.27),
                        layers.BatchNormalization(),
                        layers.Dense(units=1024,activation='relu'),
                        layers.Dropout(0.27),
                        layers.BatchNormalization(),
                        layers.Dense(units=1,activation='sigmoid')])

In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop=callbacks.EarlyStopping(min_delta=0.001, patience=20, restore_best_weights=True)
history = model.fit(
    xtr, ytr,
    validation_data=(xte, yte),
    batch_size=512,
    epochs=100,
    callbacks=[early_stop]
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

## 87.6% Accuracy